# Laboratorio De Reposición

In [67]:
import numpy as np
from math import *
import random as rd
import pandas as pd

Leer el archivo CSV proporcionado y almacenarlo en un np.array para ser trabajado en el notebook.

In [68]:
data = pd.read_csv("./framingham.csv")

Ajustar un modelo logístico polinomial en base al juego de datos cargado de forma matricial que relaciona las
variables independientes que usted considere apropiadas (puede no utilizar todas las componentes de X), con la
variable dependiente de salida (sufre o no sufre un paro cardíaco)

Para comenzar se va a ver si las variables estan normalizadas.


In [69]:
data = data.dropna()
print(data["TenYearCHD"].value_counts())

0    3099
1     557
Name: TenYearCHD, dtype: int64


Como se puede ver, las variables no estan balanceadas, por lo que se va a hacer oversampling para balancearlas.

In [70]:
from imblearn.over_sampling import RandomOverSampler

X = data.drop("TenYearCHD", axis=1)
y = data["TenYearCHD"]

oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X, y)

print(y_resampled.value_counts())

0    3099
1    3099
Name: TenYearCHD, dtype: int64


Se van a normalizar las variables para que no haya problemas con el modelo.

In [71]:
# Se normalizan los datos utilizando StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_resampled)

# transformar los datos normalizados
X_norm = scaler.transform(X_resampled)


Ahora con las variables balanceadas, se van a seleccionar las variables que se van a utilizar para el modelo.

In [72]:

from sklearn.decomposition import PCA

n = 8
pca = PCA(n_components=n)
X_pca = pca.fit_transform(X_resampled)

# obtener la matriz de diseño transformada con las mismas columnas que las características originales
X_transformed = pca.transform(X_norm)

# seleccionar solo las primeras 5 columnas de X_resampled
X_resampled_5cols = X_resampled.iloc[:, :n]

X_transformed = pd.DataFrame(X_transformed, columns=X_resampled_5cols.columns)


# imprimir los nombres de las características después del PCA
print("Características después del PCA:")
print(X_transformed.columns)

Características después del PCA:
Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp'],
      dtype='object')


c:\Users\Juan_Avila\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


Se separan los datos de entrenamiento y de prueba.

In [73]:
# se separan los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_resampled, test_size=0.2, random_state=0)

Se crean las variables de forma polinomial y se ajusta un modelo logístico de regresión.

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, accuracy_score


# se hace grid search para encontrar el mejor grado de polinomio

best_accuracy = 0
best_degree = 0

for degree in range(1, 6):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)

    # ajustar un modelo logístico con características polinomiales
    logreg = LogisticRegression(max_iter=10000)
    logreg.fit(X_train_poly, y_train)

    y_pred = logreg.predict(X_test_poly)
    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_degree = degree

poly = PolynomialFeatures(degree=best_degree, include_bias=False)
X_poly = poly.fit_transform(X_transformed)

# ajustar un modelo logístico con características polinomiales
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_poly, y_resampled)

y_pred = logreg.predict(X_poly)
accuracy = accuracy_score(y_resampled, y_pred)

print("El polinomio de grado", best_degree," con un accuracy de: ", accuracy)

El polinomio de grado 2  con un accuracy de:  0.6624717650855114


Se realiza cross validation para ver que tan bueno es el modelo.

In [75]:
# Se hace cross validation con el modelo
from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg, X_poly, y_resampled, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.66 (+/- 0.02)


# Hallazgos

Al elegir la variable objetivo 'TenYearCHD' se encontraron los valores desbalanceados por lo que se tuvo que realizar overfitting para balancear los datos. Se utilizó la librería imblearn para realizar el overfitting.

Asimismo, se tuvo que balancear los datos para poder realizar el PCA ya que si no se balancean los datos, el PCA no funciona correctamente y el modelo tampoco va a poder hacer buenas predicciones.

Con lo que respecta a las columnas a utilizar se utilizo PCA con 8 columnas pero al alterar ese valor se puede observar que varia el accuracy del modelo.
Se utilizó un polinomio de grado 2 para el modelo logístico y se obtuvo un accuracy de 0.66. Al realizar cross validation se obtuvo un accuracy de 0.66.
Por ultimo se puede observar que el modelo logístico no es el mejor para este tipo de datos ya que el accuracy es bajo, pero esto puede suceder por la cantidad de variables que se estan utilizando en el PCA y el polinomio elegido. Si se modifican estos valores pueden cambiar los resultados.